In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torchsummary as ts


class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        # cnn layers
        nn.Conv2d()
        
        self.conv1 = nn.Conv2d(i3, 32, 3, stride = 2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, stride = 2, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        
        # batch normalization
        self.batch1 = nn.BatchNorm2d(32)
        self.batch2 = nn.BatchNorm2d(64)
        self.batch3 = nn.BatchNorm2d(64)
        self.batch4 = nn.BatchNorm2d(128)
        
        
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        #fully connected layer
        self.fc1 = nn.Linear(4608, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 133)
        
        #drop out layer
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        # save the batch size of input tensor
        shape = x.size(0)
        # pass input tensor to the cnn layers
        #in the first layer pooling is not used, its show improvement accuracy
        x = F.relu(self.batch1(self.conv1(x)))
        x = self.pool(F.relu(self.batch2(self.conv2(x))))
        x = self.pool(F.relu(self.batch3(self.conv3(x))))
        x = self.pool(F.relu(self.batch4(self.conv4(x))))
        # flatten image input
        x = x.view(shape, -1)
        # pass to fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        return self.fc3(x)
    
    
model = Net()
ts.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             896
       BatchNorm2d-2         [-1, 32, 112, 112]              64
            Conv2d-3           [-1, 64, 56, 56]          18,496
       BatchNorm2d-4           [-1, 64, 56, 56]             128
         MaxPool2d-5           [-1, 64, 28, 28]               0
            Conv2d-6           [-1, 64, 26, 26]          36,928
       BatchNorm2d-7           [-1, 64, 26, 26]             128
         MaxPool2d-8           [-1, 64, 13, 13]               0
            Conv2d-9          [-1, 128, 13, 13]          73,856
      BatchNorm2d-10          [-1, 128, 13, 13]             256
        MaxPool2d-11            [-1, 128, 6, 6]               0
           Linear-12                  [-1, 512]       2,359,808
          Dropout-13                  [-1, 512]               0
           Linear-14                  [

In [ ]:
import tqdm

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in tqdm(enumerate(loaders['train'])):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass
            ouput = model(data)
            # calculate loss
            loss = criterion(ouput, target)
            #backward pass: backpropagation
            loss.backward()
            # update weight
            optimizer.step()
            # claulate train loss
            train_loss += loss.item() * data.size(0)
            
            
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
             # forward pass
            output = model(data)
            # calculate loss
            loss_p = criterion(output, target)
            # update validation loss
            valid_loss += loss_p.item() * data.size(0)
            # calculate accuracy
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tAcc:{:.2f}'.format(
            epoch, 
            train_loss,
            valid_loss,
            (correct / total * 100)))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('\tValidation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model


In [ ]:
The output is better than I expected

1.   As validation loss is still decreasing, so more epoch mayebe inprove results
2.  Add more fully connected layers
3.  For human detection, use more complex model